Tracking Objects in Video with Particle Filters
===============================================

Import libraries
to locate and track the oerson walking is challenging problem 
it was shot with a digital camera pixel values are noisy 
camera direction is also constantly changing 
and her profile keep changing too as she walks
plus she steps behind a treee 


In [68]:
import numpy as np
import cv2
#PARTICLE FILTERS ARE ALSO A POWERFUL EFFICIENT SOLUTION ROBUTS FOR ROBOTICS AI EVEN FINANCE 
# Repeatability
np.random.seed(0)

VFILENAME = "walking.mp4"
HEIGHT = 406
WIDTH = 722

Load video frames from file

In [69]:
def get_frames(filename):
    video= cv2.VideoCapture(filename) #going to return a video capture object 
    while video.isOpened(): #as long as video opened we want to read the frames 
        ret,frame= video.read()
        #goign to return like a status flag and the frame as long as the frame is valid 
        #python generator to build frames 
        if ret :
            yield frame
        else: 
            break 
    video.release()
    yield None
    return []

Creating a particle cloud

In [70]:
NUM_PARTICLES=500
#SAME RESULS AS WITH 150 PRETTY IMPRESSIVE 
#EFFECTIVE ON MUCH SMALLER COMPUTING DEVICES SMALL DIGITAL CAMERAS 
VEL_RANGE = 0.5
def initialize_particles():
    particles=np.random.rand(NUM_PARTICLES,4)
    particles = particles *  np.array((WIDTH,HEIGHT,VEL_RANGE,VEL_RANGE))
    #want the velocitiie to be centered at zero 
    #shoft last two columsn down a bit 
    particles[:,2:4] -=VEL_RANGE/2.0
    print(particles[:20,:])
    return particles
#may notice that they aren't moving even if they have a velocity 
#we aren't updating their posititons according to the velocities yet 

Moving particles according to their velocity state

In [71]:
def apply_velocity(particles):
    particles[:,0]+=particles [:,2]
    particles[:,1]+=particles [:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [72]:
def enforce_edges(particles):
    for i in range(NUM_PARTICLES):
        particles[i,0] = max(0, min(WIDTH-1, particles[i,0]))
        particles[i,1] = max(0, min(HEIGHT-1, particles[i,1]))
    return particles
            #X DOESN'T BECOME BEIGGER THAN WIDTH 
            
            #Y SAME THING 
            #PARTICLES THA TMOVE AROUND ANF CLEAVER ENOUGH TO NOT MOVE OUT OF THE VIDEO FRMAE 
            #TO IMPROVE STATE ESTIMATION WANNA CHECK THE COLOR OF EACH PIXEL SANDING AROUND EACH PARTICLE
            #AND COMPATE IT TO THE TARGET COLOR 
            
            
            


Measure each particle's quality

In [73]:
def compute_errors(particles, frame):
    errors = np.zeros(NUM_PARTICLES)
    TARGET_COLOUR = np.array( (189,105,82) ) # Blue top sleeve pixel colour
#    TARGET_COLOUR = np.array( (148, 73, 49) ) # Blue top sleeve pixel colour
    for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y = int(particles[i,1])
        pixel_colour = frame[ y, x, : ]
        errors[i] = np.sum( ( TARGET_COLOUR - pixel_colour )**2 ) # MSE in colour space
    return errors

Assign weights to the particles based on their quality of match

In [74]:
def compute_weights(errors):
    #INVERT THE ERRORS HIGEHEST ERRORS FOUND  SUBSTRACT OFF THEERRORS ARRAY 
    weights =  np.max(errors)-errors
    weights[
        (particles[:,0]==0) |
        (particles[:,0]==WIDTH-1) | 
        (particles[:,0]==0) | 
        (particles[:,0]==HEIGHT-1)
    ]=0.0
    weights= weights**6
    #POWER FOUR WORKS BETTER BUT NOT GO TOO HIGH OTHERWISE BECORME TOO SENSITIVE TO COLOUR 
    #AND  CHNAGE IN BRIGHTNESS MAY EFFECT THE RESULT MORE 
    #VELOCITY PARTICLES THAT HAVE A SIMILAR VELOCITY TO THE TARGET MANAGED TO STAY ON TOP OF IT AND GOT RESSAMPELD MUCH MORE OFTNE 
    #BY THE TIME WE GET TO TH ETREE PARTICLES ON AVERAGE ARE MOVING LIKE THE TARGET 
    #WHAT HEPPENS IF WE TURN OFF THE VELOCITY (COMMENT VELOCITY LINE )
    #AFTER THE TREE IT ODESN(T FELLOW THE TARGET ANYMORE AND IT IS LESS SPREADED IN GENERAL )
    #VELOCITY MAKE THE MODEL MORE ROBUST 
    return weights

Resample particles according to their weights

In [75]:
def resample(particles, weights):
    #IF WE NORMALIZE  WEIGHTS THEM SO THAT THEY SUM TO ONE 
    #WE CAN USE THEM AS A PROBABILITY DISTRIBUTION OVER THE PARTICLES 
    probabilities= weights/np.sum(weights)
    #BUILD A NEW PARTICLE ARRAY 
    #HIGH WEIGHT CHOOSEN MANY TIMES 
    #LOW WEIGHT MIGHT NOT BE CHOOSEN AT ALL 
    index_numbers=np.random.choice(NUM_PARTICLES,
                                   size=NUM_PARTICLES,
                                   p=probabilities)
    #REBUILD PARTICLE ARRYA BASED ON INDEX LEVELS 
    particles = particles[index_numbers,:]
    #SINGLE XY POSITION BEST GUESS OF TARGET POSITION 
    x=np.mean(particles[:,0])
    y=np.mean(particles[:,1])
    return particles, (int(x),int(y))

Fuzz the particles

In [76]:
POS_SIGMA =1.0 #10 GOOD RESULTS EVEN IF THOSE VALEUS 
VEL_SIGMA =0.5 #2 GOOD RESULT EVEN WITH THIS VALUE 
#GREATE BECAUSE IF OUR MODEL WAS BASED ESPECIALLY TUNED PARAMTERS IT WOULD NOT BE SO MUCH ROBUST TO SET UP 
def apply_noise(particles):
    #HOW BIG CAN THE STD BE AND STILL THE PARTICLE FELLOW TARGET ? 
    #STANDARD DEVIATION OG THE GAUSSIAN WE USED ARE WHAT CONTROLLS THE AMOUNT OF NOISE 
    #ADD GAUSSIAN NOISE TO EACH PARTICLE 
    #IF THE TARGET CHANGES IN NEXT FRAME SOME OF THE PARTICLES WOULD HAVE CHANGED IN THE SAME WAY 
    #THANKS TO THE VARIATION WITH THE NOISE WE ADDED SO THEY MOVE ALONG WITH THE TARGET 
    #MORE COLOUR ERROR FOR THE PARTICLES THAT DIDN T MOVE WITH THE TARGET + WON'T GET RESSAMPLED 
    noise = np.concatenate(
     ( np.random.normal(0.0,POS_SIGMA,(NUM_PARTICLES,1)),
       np.random.normal(0.0,POS_SIGMA,(NUM_PARTICLES,1)),
       np.random.normal(0.0,VEL_SIGMA,(NUM_PARTICLES,1)),
       np.random.normal(0.0,VEL_SIGMA,(NUM_PARTICLES,1))
     ),
        axis=1
    )
    particles += noise
    #WE NEED TO MAKE WEIGHTS THEM MORE SENSITIVE TO COLOR DIFFERENCE 
    #SQUARE THE WEIGHTS 
    return particles

Display the video frames

In [77]:
#display those video frame  by one 
def display(frame, particles, location):
    if len(particles)>0:
        for i in range(NUM_PARTICLES):
            x=int(particles[i,0])
            y=int(particles[i,1])
            cv2.circle(frame,(x,y),1 ,(0,255,0),1)
                #gree, circle small curve thickness = one
    if len(location)>0  : 
        cv2.circle(frame,location ,15, (0,0,255),5)
        
                
            
    #display video frame 
    cv2.imshow('frame',frame )
    #the first argument is the window and we handled it the video frame 
    if cv2.waitKey(30) == 27: #wait for thirty ms to see if user press escape key
        #if they did 
        if cv2.waitKey(0) == 27: # zero means wait indefinite amount of time 
            return True #to let the program know that the program should be stopped 
    return False

Main routine

In [78]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()


[[ 3.96243350e+02  2.90366883e+02  5.13816880e-02  2.24415915e-02]
 [ 3.05878765e+02  2.62233010e+02 -3.12063944e-02  1.95886500e-01]
 [ 6.95764513e+02  1.55677257e+02  1.45862519e-01  1.44474599e-02]
 [ 4.10128173e+02  3.75792235e+02 -2.14481971e-01 -2.06435350e-01]
 [ 1.45976830e+01  3.38043657e+02  1.39078375e-01  1.85006074e-01]
 [ 7.06562443e+02  3.24458377e+02 -1.92603189e-02  1.40264588e-01]
 [ 8.53941355e+01  2.59807935e+02 -1.78323356e-01  2.22334459e-01]
 [ 3.76774488e+02  1.68352748e+02 -1.17722194e-01  1.37116845e-01]
 [ 3.29340540e+02  2.30784183e+02 -2.40605100e-01  5.88177485e-02]
 [ 4.41933112e+02  2.50475203e+02  2.21874039e-01  9.09101496e-02]
 [ 2.59564704e+02  1.77434973e+02  9.88155980e-02 -2.19887264e-01]
 [ 4.81405569e+02  2.72278975e+02 -1.44808719e-01 -1.85536851e-01]
 [ 2.27739269e+02  1.47666573e+02  3.50983852e-02 -3.06992433e-02]
 [ 7.13605911e+02  4.14301932e+01 -1.45561622e-01 -1.69345241e-01]
 [ 4.71544211e+02  1.02836391e+02 -1.68446136e-02 -1.27787204e